In [5]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

## production Planning
An automobile assembly plant assembles two types of vehicles: a four-door saloon and a people carrier. Both vehicle types must pass through a painting plant and an assembly plant. If the painting plant only paints four-door saloons, it can paint some 2,000 vehicles each day, whereas if it paints only people carriers, it can paint some 1,500 vehicles each day. Moreover, if the assembly plant only assembles either four-door saloons or people carriers, it can assemble some 2,200 vehicles every day. Each people carrier implies an average profit of $3,000, whereas a fourdoor saloon implies an average profit of $2,100. maximise the vehicle assembly plant’s daily profit

In [6]:
#instatiate an object of ConcreteModel class
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=range(1, 3))

# Parameters

# painting_parameter
model.pa = pyo.Param(model.i, initialize = {1:2000,2:1500})
pa = model.pa

# assembly_parameter
model.a = pyo.Param(model.i, initialize = {1:2200,2:2200})
a = model.a

#profit_parameter
model.p = pyo.Param(model.i, initialize = {1:2100,2:3000})
p = model.p

# Decision Variables

model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)
x= model.x


#Objective Function
def daily_profit(model):
    return sum(p[i] * x[i] for i in model.i)


model.obj = pyo.Objective(rule=daily_profit, sense=pyo.maximize)

# Constraints
def painting_constraint(model,i):
    """fraction of painting for each car type must be less than or equal to 1

    Args:
        model (_type_): _description_
        i (_type_): _description_
    """
    return sum((1/pa[i])*x[i] for i in model.i) <= 1

def assembly_constraint(model,i):
    """fraction of assembly for each car type must be less than or equal to 1

    Args:
        model (_type_): _description_
        i (_type_): _description_
    """
    return sum((1/a[i])*x[i] for i in model.i) <= 1

model.const1 = pyo.Constraint(model.i, rule=painting_constraint)
model.const2 = pyo.Constraint(model.i, rule=assembly_constraint)

# Solve the model

solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)
print("PROFIT = ",model.obj())
for i in model.i:
    print("value of ", i," =",x[i]() )
    


Problem: 
- Name: unknown
  Lower bound: 4500000.0
  Upper bound: 4500000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Number of nonzeros: 8
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.002629995346069336
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

PROFIT =  4500000.0
value of  1  = 0.0
value of  2  = 1500.0


## Portfolio Manager

ORGASA has a portfolio of investments in shares, bonds and other investment alternatives. Now it has available funds amounting to $200,000 which must be considered for new investments. The four investment alternatives that ORGASA is contemplating are offered in Table
The measure of risk indicates uncertainty associated with the share in terms of its capacity to reach the annual return foreseen; the higher the value, the greater the risk. ORGASA has stipulated the following conditions for its investment: 

![Table](image.png)


• Rule 1: The annual rate of return from this portfolio must be at least 9 %. 
• Rule 2: No value can represent more than 50 % of the total investment in dollars. 

(a) Use the linear programming model to develop a portfolio of investments which minimises the risk. 
(b) If the firm ignores the risk involved and uses a maximum return strategy on its investment, how will the former model be amended

In [7]:
#instatiate the model from ConcretModel
model = pyo.ConcreteModel()

# Set
model.i = pyo.Set(initialize = range(1,5))


# Parameters
model.p = pyo.Param(model.i, initialize = {1:100,2:50,3:80,4:40})
p = model.p # price per share

model.ar = pyo.Param(model.i, initialize ={1:0.12,2:0.08,3:0.06,4:0.1})
ar = model.ar # annual rate of return

model.r = pyo.Param(model.i, initialize ={1:0.1,2:0.07,3:0.05,4:0.08})
r = model.r # measure of risk on per dollar of invested



# Variables

def bounds(model,i):
    lb ={}
    ub = {}
    for i in model.i:
        lb[i] = 0
        ub[i] = 200000*0.5/p[i]
    return (lb[i],ub[i])

model.x = pyo.Var(model.i, within = pyo.NonNegativeReals, bounds=bounds) 
x=  model.x

# objective function
def minimize_risk(model):
    return sum(r[i]*p[i]*x[i] for i in model.i)

model.obj2 = pyo.Objective(rule = minimize_risk, sense = pyo.minimize)

# Constraints

def aror_constraint(model,i):
    # annual rate of return constraint atleast 9%
    return sum(ar[i]*p[i]*x[i] for i in model.i) >= 200000 *0.09

model.const1 = pyo.Constraint(model.i, rule = aror_constraint)


## either use below code or use the bounds
# def constr2(model,i):
#     return p[1]*x[1]<=200000 *0.5

# model.const2 = pyo.Constraint(model.i, rule = constr2) 
# def constr3(model,i):
#     return p[2]*x[2]<=200000 *0.5

# model.const3 = pyo.Constraint(model.i, rule = constr3) 

# def constr4(model,i):
#     return p[3]*x[3]<=200000 *0.5

# model.const4 = pyo.Constraint(model.i, rule = constr4) 
# def constr5(model,i):
#     return p[4]*x[4]<=200000 *0.5

# model.const5 = pyo.Constraint(model.i, rule = constr5) 

def equality_investment(model,i):
    return sum(p[i]*x[i] for i in model.i) == 200000

model.const6 = pyo.Constraint(model.i, rule = equality_investment)

# Solve the model


solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)
print("PROFIT = ",model.obj2())
for i in model.i:
    print("value of ", i," =",x[i]() )
    


Problem: 
- Name: unknown
  Lower bound: 14666.6666666667
  Upper bound: 14666.6666666667
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 4
  Number of nonzeros: 32
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004260540008544922
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

PROFIT =  14666.666666666662
value of  1  = 333.333333333333
value of  2  = 0.0
value of  3  = 833.333333333333
value of  4  = 2500.0


In [8]:
#instatiate the model from ConcretModel
model = pyo.ConcreteModel()

# Set
model.i = pyo.Set(initialize = range(1,5))


# Parameters
model.p = pyo.Param(model.i, initialize = {1:100,2:50,3:80,4:40})
p = model.p # price per share

model.ar = pyo.Param(model.i, initialize ={1:0.12,2:0.08,3:0.06,4:0.1})
ar = model.ar # annual rate of return

model.r = pyo.Param(model.i, initialize ={1:0.1,2:0.07,3:0.05,4:0.08})
r = model.r # measure of risk on per dollar of invested



# Variables

def bounds(model,i):
    lb ={}
    ub = {}
    for i in model.i:
        lb[i] = 0
        ub[i] = 200000*0.5/p[i]
    return (lb[i],ub[i])

model.x = pyo.Var(model.i, within = pyo.NonNegativeReals, bounds=bounds) 
x=  model.x

# objective function
def maximize_profit(model):
    return sum(r[i]*p[i]*x[i] for i in model.i)

model.obj2 = pyo.Objective(rule = maximize_profit, sense = pyo.maximize)

# Constraints

def aror_constraint(model,i):
    # annual rate of return constraint atleast 9%
    return sum(ar[i]*p[i]*x[i] for i in model.i) >= 200000 *0.09

model.const1 = pyo.Constraint(model.i, rule = aror_constraint)


## either use below code or use the bounds
# def constr2(model,i):
#     return p[1]*x[1]<=200000 *0.5

# model.const2 = pyo.Constraint(model.i, rule = constr2) 
# def constr3(model,i):
#     return p[2]*x[2]<=200000 *0.5

# model.const3 = pyo.Constraint(model.i, rule = constr3) 

# def constr4(model,i):
#     return p[3]*x[3]<=200000 *0.5

# model.const4 = pyo.Constraint(model.i, rule = constr4) 
# def constr5(model,i):
#     return p[4]*x[4]<=200000 *0.5

# model.const5 = pyo.Constraint(model.i, rule = constr5) 

def equality_investment(model,i):
    return sum(p[i]*x[i] for i in model.i) == 200000

model.const6 = pyo.Constraint(model.i, rule = equality_investment)

# Solve the model


solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)
print("PROFIT = ",model.obj2())
for i in model.i:
    print("value of ", i," =",x[i]() )
    


Problem: 
- Name: unknown
  Lower bound: 20000.0
  Upper bound: 20000.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 4
  Number of nonzeros: 32
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005602121353149414
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

PROFIT =  20000.0
value of  1  = 2000.0
value of  2  = 0.0
value of  3  = 0.0
value of  4  = 0.0
